In [59]:
import numpy as np
from scipy.stats import mode
import time

import warnings
warnings.filterwarnings('ignore')

In [60]:
X = np.genfromtxt("data/data.csv", delimiter=',')
y = np.genfromtxt("data/label.csv", delimiter=',')

In [61]:
np.random.seed(42)

In [62]:
def euclidean_distance(x1, x2):
    euclidean_distance = np.sqrt(np.sum((x1 - x2) ** 2))
    return euclidean_distance

def cosine_distance(x1, x2):
    dot_product = np.dot(x1, x2)
    norm_x1 = np.linalg.norm(x1)
    norm_x2 = np.linalg.norm(x2)
    cosine_similarity = dot_product / (norm_x1 * norm_x2)
    cosine_distance = 1 - cosine_similarity
    return cosine_distance

def jaccard_distance(x1, x2):
    jaccard_similarity = np.sum(np.minimum(x1, x2)) / np.sum(np.maximum(x1, x2))
    jaccard_distance = 1 - jaccard_similarity
    return jaccard_distance

In [63]:
def assign_majority_labels(labels, y, k):
    majority_labels = np.zeros(10000)
    for cluster in range(k):
        cluster_indices = np.where(labels == cluster)
        cluster_actual_labels = y[cluster_indices]
        majority_label = mode(cluster_actual_labels)[0][0]
        majority_labels[cluster_indices] = majority_label
    return majority_labels

def calculate_accuracy(labels, y, k=10):
    pred_labels = assign_majority_labels(labels, y, 10)
    accuracy = np.sum(pred_labels == y) / len(y)
    return accuracy 

def calculate_sse(X, labels, centroids):
    sse = np.sum(np.sum((X[labels == i] - centroids[i])**2) for i in range(len(centroids)))
    return sse

In [64]:
def k_means_plusplus_centroids_init(X, distance_function=euclidean_distance, k=10):
    centroids = []
    centroids.append(X[np.random.choice(len(X))])
    for _ in range(1, k):
        distances = np.array([min(distance_function(x, c)**2 for c in centroids) for x in X])
        probabilities = distances / np.sum(distances)
        new_centroid_index = np.random.choice(len(X), p=probabilities)
        centroids.append(X[new_centroid_index])
    return np.array(centroids)

def k_means_centroids_init(X, k=10):
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]
    return centroids

def k_means(X, centroids, distance_function=euclidean_distance, stop_criteria="max_iters_complete", k=10, max_iters=200):
    labels = None
    itr = 0
    prev_sse, cur_sse = float('inf'), float('inf')
    while itr < max_iters:
        labels = np.argmin(np.array([[distance_function(x, c) for c in centroids] for x in X]), axis=1)
        new_centroids = np.array([X[labels == i].mean(axis=0) if np.sum(labels == i) > 0 else centroids[i] for i in range(k)])
        prev_sse = cur_sse
        cur_sse = calculate_sse(X, labels, new_centroids)
        itr += 1
        if stop_criteria == "check_centroids":
            if np.array_equal(centroids, new_centroids):
                break
        elif stop_criteria == "sse_increase":
            if prev_sse <= cur_sse:
                break
        centroids = new_centroids
    return labels, centroids, itr

In [65]:
distance_function_list = [euclidean_distance, cosine_distance, jaccard_distance]
stop_criteria_list = ["check_centroids", "sse_increase", "max_iters_complete"]

In [66]:
# Question 1, 2, 3, 4

for distance_function in distance_function_list:
    init_centroids = k_means_plusplus_centroids_init(X, distance_function)
    for stop_criteria in stop_criteria_list:
        start_time = time.time()
        labels, centroids, itr = k_means(X, init_centroids, distance_function, stop_criteria)
        end_time = time.time()
        sse = calculate_sse(X, labels, centroids)
        accuracy = calculate_accuracy(labels, y) * 100
        elapsed_time = end_time - start_time
        print("({}, {}) - SSE: {:.2f}, Accuracy: {:.2f}, iterations: {}, time_taken_in_seconds: {:.2f}".format(str(distance_function).split()[1], stop_criteria, sse, accuracy, itr, elapsed_time))

(euclidean_distance, check_centroids) - SSE: 25433212639.24, Accuracy: 60.44, iterations: 81, time_taken_in_seconds: 56.50
(euclidean_distance, sse_increase) - SSE: 25433212639.24, Accuracy: 60.44, iterations: 81, time_taken_in_seconds: 56.20
(euclidean_distance, max_iters_complete) - SSE: 25433212639.24, Accuracy: 60.44, iterations: 200, time_taken_in_seconds: 139.57
(cosine_distance, check_centroids) - SSE: 25419566794.00, Accuracy: 61.33, iterations: 71, time_taken_in_seconds: 53.31
(cosine_distance, sse_increase) - SSE: 25440247462.49, Accuracy: 61.16, iterations: 16, time_taken_in_seconds: 11.90
(cosine_distance, max_iters_complete) - SSE: 25419566794.00, Accuracy: 61.33, iterations: 200, time_taken_in_seconds: 148.57
(jaccard_distance, check_centroids) - SSE: 25417331725.24, Accuracy: 60.33, iterations: 80, time_taken_in_seconds: 77.25
(jaccard_distance, sse_increase) - SSE: 25487498983.53, Accuracy: 60.89, iterations: 18, time_taken_in_seconds: 17.55
(jaccard_distance, max_iters

In [67]:
# Testing with sklearn library
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
sse = calculate_sse(X, labels, centroids)
accuracy = calculate_accuracy(labels, y)
print("SSE: {:.2f}, Accuracy: {:.2f}".format(sse, accuracy * 100))

SSE: 25320432712.92, Accuracy: 59.48
